In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
   for filename in filenames:
      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e24/sample_submission.csv
/kaggle/input/playground-series-s3e24/train.csv
/kaggle/input/playground-series-s3e24/test.csv


In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [4]:
train = pd.read_csv("/kaggle/input/playground-series-s3e24/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s3e24/test.csv")

In [5]:
combine_df = [train,test]
combine_df

[            id  age  height(cm)  weight(kg)  waist(cm)  eyesight(left)  \
 0            0   55         165          60       81.0             0.5   
 1            1   70         165          65       89.0             0.6   
 2            2   20         170          75       81.0             0.4   
 3            3   35         180          95      105.0             1.5   
 4            4   30         165          60       80.5             1.5   
 ...        ...  ...         ...         ...        ...             ...   
 159251  159251   40         155          45       69.0             1.5   
 159252  159252   50         155          75       82.0             1.0   
 159253  159253   40         160          50       66.0             1.5   
 159254  159254   50         165          75       92.0             1.2   
 159255  159255   40         145          45       76.4             1.0   
 
         eyesight(right)  hearing(left)  hearing(right)  systolic  ...  HDL  \
 0                 

In [6]:
train = train.drop(["id","hearing(left)","hearing(right)","AST"],axis=1)
test = test.drop(["hearing(left)","hearing(right)","AST"],axis=1)
combine_df = [train,test]

In [7]:
train.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,ALT,Gtp,dental caries,smoking
0,55,165,60,81.0,0.5,0.6,135,87,94,172,300,40,75,16.5,1,1.0,25,27,0,1
1,70,165,65,89.0,0.6,0.7,146,83,147,194,55,57,126,16.2,1,1.1,23,37,1,0
2,20,170,75,81.0,0.4,0.5,118,75,79,178,197,45,93,17.4,1,0.8,31,53,0,1
3,35,180,95,105.0,1.5,1.2,131,88,91,180,203,38,102,15.9,1,1.0,27,30,1,0
4,30,165,60,80.5,1.5,1.0,121,76,91,155,87,44,93,15.4,1,0.8,13,17,0,1


In [8]:
train['AgeBand'] = pd.cut(train['age'], 5)
train[['AgeBand', 'smoking']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,smoking
0,"(19.935, 33.0]",0.551895
1,"(33.0, 46.0]",0.487106
2,"(46.0, 59.0]",0.368621
3,"(59.0, 72.0]",0.278552
4,"(72.0, 85.0]",0.140049


In [9]:
for dataset in combine_df:
    dataset.loc[(dataset["age"]>19) & (dataset["age"]<=33),"age"] = 0
    dataset.loc[(dataset["age"]>33) & (dataset["age"]<=46),"age"] = 1
    dataset.loc[(dataset["age"]>46) & (dataset["age"]<=59),"age"] = 2
    dataset.loc[(dataset["age"]>59) & (dataset["age"]<=72),"age"] = 3
    dataset.loc[dataset["age"]>72,"age"] = 4
train.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),systolic,relaxation,fasting blood sugar,Cholesterol,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,ALT,Gtp,dental caries,smoking,AgeBand
0,2,165,60,81.0,0.5,0.6,135,87,94,172,...,40,75,16.5,1,1.0,25,27,0,1,"(46.0, 59.0]"
1,3,165,65,89.0,0.6,0.7,146,83,147,194,...,57,126,16.2,1,1.1,23,37,1,0,"(59.0, 72.0]"
2,0,170,75,81.0,0.4,0.5,118,75,79,178,...,45,93,17.4,1,0.8,31,53,0,1,"(19.935, 33.0]"
3,1,180,95,105.0,1.5,1.2,131,88,91,180,...,38,102,15.9,1,1.0,27,30,1,0,"(33.0, 46.0]"
4,0,165,60,80.5,1.5,1.0,121,76,91,155,...,44,93,15.4,1,0.8,13,17,0,1,"(19.935, 33.0]"


In [10]:
train = train.drop("AgeBand",axis=1)
combine_df=[train,test]

Create a BMI variable

In [11]:
for dataset in combine_df:
    dataset["BMI"] = dataset["weight(kg)"] / (dataset["height(cm)"] * 0.01)
    

In [12]:
train = train.drop(["weight(kg)","height(cm)"],axis=1)
test = test.drop(["weight(kg)","height(cm)"],axis=1)
train.head()

,age,waist(cm),eyesight(left),eyesight(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,ALT,Gtp,dental caries,smoking,BMI
0,2,81.0,0.5,0.6,135,87,94,172,300,40,75,16.5,1,1.0,25,27,0,1,36.363636
1,3,89.0,0.6,0.7,146,83,147,194,55,57,126,16.2,1,1.1,23,37,1,0,39.393939
2,0,81.0,0.4,0.5,118,75,79,178,197,45,93,17.4,1,0.8,31,53,0,1,44.117647
3,1,105.0,1.5,1.2,131,88,91,180,203,38,102,15.9,1,1.0,27,30,1,0,52.777778
4,0,80.5,1.5,1.0,121,76,91,155,87,44,93,15.4,1,0.8,13,17,0,1,36.363636


In [13]:
for dataset in combine_df:
    dataset["eyesight"] = dataset["eyesight(left)"] + dataset["eyesight(right)"]   

In [14]:
train = train.drop(["eyesight(left)","eyesight(right)"],axis=1)
test = test.drop(["eyesight(left)","eyesight(right)"],axis=1)
train.head()

,age,waist(cm),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,ALT,Gtp,dental caries,smoking,BMI
0,2,81.0,135,87,94,172,300,40,75,16.5,1,1.0,25,27,0,1,36.363636
1,3,89.0,146,83,147,194,55,57,126,16.2,1,1.1,23,37,1,0,39.393939
2,0,81.0,118,75,79,178,197,45,93,17.4,1,0.8,31,53,0,1,44.117647
3,1,105.0,131,88,91,180,203,38,102,15.9,1,1.0,27,30,1,0,52.777778
4,0,80.5,121,76,91,155,87,44,93,15.4,1,0.8,13,17,0,1,36.363636


### Model Training

In [15]:
X = train.drop("smoking",axis=1)
y = train["smoking"]

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [17]:
from sklearn.metrics import accuracy_score

In [18]:
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
yhat = logreg.predict(X_test)
# evaluate predictions
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

Accuracy: 0.731


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=3)

In [20]:
yhat = knn.predict(X_test)
acc = accuracy_score(y_test,yhat)
print('Accuracy: %.3f' % acc)

Accuracy: 0.669


In [21]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
yhat = decision_tree.predict(X_test)
acc = accuracy_score(y_test,yhat)
print('Accuracy: %.3f' % acc)

Accuracy: 0.678


In [22]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
yhat = random_forest.predict(X_test)
acc = accuracy_score(y_test,yhat)
print('Accuracy: %.3f' % acc)

Accuracy: 0.764


In [23]:
xgboost = XGBClassifier()
xgboost.fit(X_train,y_train)
yhat = xgboost.predict(X_test)
acc = accuracy_score(y_test,yhat)
print('Accuracy: %.3f' % acc)

Accuracy: 0.773


In [24]:
test.head()

,id,age,waist(cm),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,ALT,Gtp,dental caries,BMI
0,159256,1,84.0,130,89,107,200,186,49,115,14.2,1,0.9,25,32,0,42.424242
1,159257,4,93.0,144,72,93,172,158,35,104,13.0,1,1.1,12,24,0,37.500000
2,159258,3,86.5,117,75,92,161,173,39,88,15.4,1,1.4,60,36,0,41.176471
3,159259,1,67.0,116,62,91,213,47,75,128,14.5,1,0.6,18,10,1,31.250000
4,159260,1,89.4,132,94,79,170,100,39,123,16.5,1,1.0,39,27,1,44.117647


In [25]:
X_test = test.drop("id",axis=1)
X_test.head()

,age,waist(cm),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,ALT,Gtp,dental caries,BMI
0,1,84.0,130,89,107,200,186,49,115,14.2,1,0.9,25,32,0,42.424242
1,4,93.0,144,72,93,172,158,35,104,13.0,1,1.1,12,24,0,37.500000
2,3,86.5,117,75,92,161,173,39,88,15.4,1,1.4,60,36,0,41.176471
3,1,67.0,116,62,91,213,47,75,128,14.5,1,0.6,18,10,1,31.250000
4,1,89.4,132,94,79,170,100,39,123,16.5,1,1.0,39,27,1,44.117647


In [26]:
y_pred = xgboost.predict(X_test)
y_pred

array([1, 0, 1, ..., 1, 0, 0])

In [27]:
result = pd.DataFrame(
{
    "id": test["id"],
    "smoking": y_pred
})

In [28]:
result.head()

,id,smoking
0,159256,1
1,159257,0
2,159258,1
3,159259,0
4,159260,1


In [29]:
result.to_csv('/kaggle/working/submission.csv', index=False)